## Librerie e funzioni

In [12]:
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Parametri geometrici delle lastre
larghezza_pla = 26.7  # cm
lunghezza_pla = 52.5  # cm
profondità_pla = 2.3  # cm

# Distanze tra i PMT
distanza_7_4 = 28.9  # cm
distanza_4_2 = 39.4  # cm
distanza_2_b = 7  # cm

# Efficienze dei PMT
efficienze = {
    "PMT7": 0.3,
    "PMT4": 0.8,
    "PMT2": 0.3,
    "PMT8": 0.3,
    "PMT9": 0.4,
    "PMT10": 0.3,
    "PMT11": 0.4
}

# Dimensioni del bersaglio (PMT sotto il sistema principale)
larghezza_bers = 30  # cm
lunghezza_bers = 31  # cm

In [14]:
# Dimensioni di ciascun PMT nel bersaglio
larghezza_pmt_bers = larghezza_bers / 2  # ogni PMT largo metà del sistema
lunghezza_pmt_bers = lunghezza_bers / 2  # ogni PMT lungo metà del sistema
profondità_pmt_bers=30/2

offset_x=(lunghezza_pla-lunghezza_bers)/2
offset_y=(larghezza_bers-larghezza_pla)/2

# Funzione per verificare se una posizione è all'interno dei limiti geometrici
def is_inside(x, y, larghezza, lunghezza):
    return (0<= x <= larghezza) and (0 <= y <= lunghezza)

# Funzione per verificare se una posizione è all'interno dei limiti geometrici
def is_inside_target(x, y, larghezza, lunghezza):
    return (offset_x<= x <= larghezza+offset_x) and (offset_y <= y <= lunghezza+offset_y)

# Simulazione di un evento per i PMT bersaglio
def simulate_target_event(xb89, yb89,theta,phi):
    # Limiti del sistema bersaglio rispetto a PMT2    
    xb01 = xb89- profondità_pmt_bers * np.tan(theta) * np.cos(phi)
    yb01 = yb89 - profondità_pmt_bers * np.tan(theta) * np.sin(phi)

    # Coordinate dei centri dei PMT bersaglio
    pmt_positions = {
        "PMT8": (xb89,yb89),
        "PMT9": (xb89,yb89),
        "PMT10": (xb01,yb01),
        "PMT11": (xb01,yb01), 
    }

    # Verifica hit su ciascun PMT
    hit_results = []
    for pmt, (px, py) in pmt_positions.items():
        is_hit = is_inside_target(px, py, larghezza_pmt_bers, lunghezza_pmt_bers) and np.random.uniform(0, 1) < efficienze[pmt]
        hit_results.append(is_hit)

    # Logica OR tra i PMT del bersaglio
    return any(hit_results)

# Simulazione di un evento principale (PMT superiori + bersaglio)
def simulate_event():
    # Generazione casuale di theta e phi
    phi = np.random.uniform(0, 2 * np.pi)  # Distribuzione uniforme in phi
    theta = np.arccos(np.sqrt(np.random.uniform(0, 1)))  # Distribuzione cos^2(theta)

    # PMT7
    x1 = np.random.uniform(0, larghezza_pla )
    y1 = np.random.uniform(0, lunghezza_pla)
    if not is_inside(x1, y1, larghezza_pla, lunghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT7"]:
        return False

    # PMT4
    x2 = x1 - distanza_7_4 * np.tan(theta) * np.cos(phi)
    y2 = y1 - distanza_7_4 * np.tan(theta) * np.sin(phi)
    if not is_inside(x2, y2, larghezza_pla, lunghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT4"]:
        return False

    # PMT2
    x3 = x2 - distanza_4_2 * np.tan(theta) * np.cos(phi)
    y3 = y2 - distanza_4_2 * np.tan(theta) * np.sin(phi)
    if not is_inside(x3, y3, larghezza_pla, lunghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT2"]:
        return False

    # Logica AND tra i PMT superiori e il bersaglio
    x89= x3 - distanza_2_b * np.tan(theta) * np.cos(phi)
    y89 = y3 - distanza_2_b* np.tan(theta) * np.sin(phi)
    return simulate_target_event(x89, y89,theta,phi)

# Simulazione Monte Carlo
def monte_carlo_simulation(duration, area):
    total_events = int(duration * area)  # Numero totale di eventi da simulare
    coincident_events = 0
    for _ in range(total_events):
        if simulate_event():
            coincident_events += 1
    return coincident_events / duration  # Rate di coincidenza

# Esecuzione della simulazione
tempo_acquisizione = 15*60  # secondi
area_totale = larghezza_pla * lunghezza_pla  # cm²
rate_coincidenza = monte_carlo_simulation(tempo_acquisizione, area_totale)
print(f"Rate di coincidenza  AND (sistema completo) per {tempo_acquisizione} s: {rate_coincidenza:.5f} eventi/s")

Rate di coincidenza  AND (sistema completo) per 900 s: 0.84333 eventi/s


## Efficienza intrinseca del VETO

In [4]:
def errore_prodotto(x,y,dx,dy):
    return np.sqrt((y*dx)**2+(x*dy)**2)
def errore_prodotto3(x,dx, y,dy, z, dz):
    return np.sqrt((x*y*dz)**2+(x*z*dy)**2+(dx*y*z)**2)

In [ ]:
def accettanza_sistema3(length1, length2, length3, width,height1, height2):
    """Calcolo della efficienza geometrica per 3 scintillatori sovrapposti con geometria diversa.
    L'altezza è positiva se considerata dal basso verso l'alto, negativa se viceversa.
    1 e 2 rappresentano la doppia, mentre 3 rappresenta lo scintillatore di cui vogliamo calcolare l'accettanza geometrica."""
    n = 10000 #Numero di tentativi
    # Inizializza contatori
    successi = 0
    tentativi = 0
    
    # Generatore di numeri casuali
    rng = np.random.default_rng() #random seed
    
    for _ in range(n):
        # Genera coordinate x1 e y1 sul primo scintillatore
        x1 = rng.uniform(0, width)
        y1 = rng.uniform(0, length1)
        
        
        # Genera angoli ϕ e θ
        phi = rng.uniform(0, 2 * np.pi)
        theta = np.arccos(rng.uniform(0, 1)**(1/4))  # distribuito come cos3(θ)
        
        # Proiezione del fascio sul secondo scintillatore
        x2 = x1 - height1 * np.tan(theta) * np.cos(phi) 
        y2 = y1 - height1 * np.tan(theta) * np.sin(phi)

        # Proiezione del fascio sul terzo scintillatore
        x3 = x2 - height2 * np.tan(theta) * np.cos(phi) 
        y3 = y2 - height2 * np.tan(theta) * np.sin(phi)
        
        # Verifica se x2 e y2 sono compresi nelle dimensioni del secondo scintillatore
        if 0 <= x2 <= width and 0 <= y2 <= length2:
            if 0 <= x3 <= width and 0 <= y3 <= length3:
                successi += 1
            tentativi+=1 #probabilità che passi dal terzo PMT data la doppia
    
    efficienza_geometrica = successi / tentativi
    errore = np.sqrt(efficienza_geometrica * (1 - efficienza_geometrica) / tentativi)
    return efficienza_geometrica, errore

In [ ]:
larghezza_pla=26.7 #cm
dlarghezza_pla=0.1#cm
lunghezza_pla=52.5 
dlunghezza_pla=0.1
prodondità_pla=2.3
dprofondità_pla=0.1

area_pla=larghezza_pla*lunghezza_pla
darea=errore_prodotto(lunghezza_pla,dlunghezza_pla,larghezza_pla,dlarghezza_pla)

distanza_7_4=28.9 #cm
distanza_4_3=9.4 
distanza_3_2=9.4
distanza_2_b=7
distanza_b_1=15

In [ ]:
triple_doppie1=0.13
errore1=0.03
triple_doppie2=0.32
errore2=0.03
triple_doppie4=0.73
errore4=0.04
triple_doppie7=0.34
errore7=0.03

distanza_2_b+distanza_b_1
acc_1, errore_acc_1=accettanza_sistema3(lunghezza_pla,lunghezza_pla,lunghezza_pla, larghezza_pla, distanza_7_4+distanza_2_b)
print(f'Accettanza1: {acc_1}')

epsilon_intr1=triple_doppie1/acc_1
depsilon_intr1=errore_prodotto(triple_doppie1,acc_1,errore1,errore_acc_1)
print(f"Efficienza intrinseca PMT1 {epsilon_intr1:.2f} ± {depsilon_intr1:.2f}")

NameError: name 'accettanza_sistema3' is not defined

In [9]:
geom_eff1, err_geom1=accettanza_sistema2(length2,length1,width,height1)
print(f"Efficienza geometrica 1: {geom_eff1:.3f}  ± {err_geom1:.3f}")

geom_eff2, err_geom2=accettanza_sistema2(length1,length2,width,-height1)
print(f"Efficienza geometrica 2: {geom_eff2:.3f}  ± {err_geom2:.3f}")

epsilon_eff1=epsilon_intr1*geom_eff1
depsilon_eff1=errore_prodotto(epsilon_intr1,geom_eff1,depsilon_intr1,err_geom1)
print(f"Efficienza corretta per l'accettanza 1: {epsilon_eff1:.2f} ± {depsilon_eff1:.2f}")

epsilon_eff2=epsilon_intr2*geom_eff2
depsilon_eff2=errore_prodotto(epsilon_intr2,geom_eff2,depsilon_intr2,err_geom2)
print(f"Efficienza corretta per l'accettanza 2: {epsilon_eff2:.2f} ± {depsilon_eff2:.2f}")

Efficienza geometrica 1: 0.938  ± 0.002
Efficienza geometrica 2: 0.605  ± 0.005
Efficienza corretta per l'accettanza 1: 0.89 ± 0.05
Efficienza corretta per l'accettanza 2: 0.53 ± 0.03


In [10]:
epsilon_eff=epsilon_eff1*epsilon_eff2
depsilon_eff=errore_prodotto(epsilon_eff1,depsilon_eff1,epsilon_eff2,depsilon_eff2)
print(f"Efficienza Setup08: {epsilon_eff:.2f} ± {depsilon_eff:.2f}")

Efficienza Setup08: 0.47 ± 0.04


In [14]:
rate_cosmic1=cosmic*aeff*1/60
drate_cosmic1=cosmic*daeff*1/60
print(f"Rate Setup08: {epsilon_eff*rate_cosmic1} ± {errore_prodotto(rate_cosmic1, epsilon_eff, drate_cosmic1, depsilon_eff)}")

Rate Setup08: 6.081140520476354 ± 0.5236151502970052


In [17]:
rate1=4548
drate1=12
rate2=1063
drate2=6
racc=rate1*rate2*2*(40e-9-2e-9)
dracc=np.sqrt((drate1*rate2)**2+(rate1*drate2)**2)*2*(40e-9-2e-9)
print(f"Rate eventi accidentali: {racc*1e3:.0f}± {dracc*1e3:.0f} mHz")

Rate eventi accidentali: 367± 2 mHz
